### Inicio de la sesion de Spark

Al no estar en un **cluster EMR**, hay que utilizar pyspark de manera local en el notebook de SageMaker. 
Por ello se tiene que ejecutar el siguiente script:


In [ ]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

# Hadoop Configuration
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.us-east-1.amazonaws.com')

### Librerias para manejo de Dataframes

In [ ]:
import pyspark.sql.functions as f

### Lectura de archivos parquet

Considerar que el role con el que se creo el notebook de SageMaker es *arn:aws:iam::639556434474:role/BigData-SageMakerCrossAccountS3-Role*, solo con ese role se podra leer los archivos en QAS

In [ ]:
df = spark.read.csv('s3a://bpc2108ch-datalke/work/covid/covid_covid/')
df.show()

In [ ]:
df = spark.read.csv('s3a://bpc2108ch-datalke/work/covid/covid_covid/') \
    .filter((f.col('pt_date') == '2021_8_22') & (f.col('pt_time') == '18_3_25'))
df.show()

In [ ]:
df = df.withColumnRenamed('_c0', 'fecha')
df = df.withColumnRenamed('_c1', 'confirmados')
df = df.withColumnRenamed('_c2', 'fallecidos')
df = df.withColumnRenamed('_c3', 'recuperados')
df = df.withColumnRenamed('_c4', 'pais')

In [ ]:
df.show()

### Consultoras activas que pasaron pedido

In [ ]:
df.filter(f.col('pais') == 'Peru').groupBy('pais','fecha')\
    .agg(f.max('confirmados')).show()